In [ ]:
import json
import os
import numpy
import pandas
import sys
import logging
import importlib

from datetime import date
from os import path
from dotenv import load_dotenv

sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from bespoke.inventory.analysis.shared import download_util
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util
from bespoke.inventory.analysis import inventory_cogs_util as cogs_util
from bespoke.inventory.analysis import inventory_summary_util
from bespoke.inventory.analysis.shared.inventory_types import (
  AnalysisSummaryDict
)

load_dotenv(verbose=True)
engine = download_util.get_bigquery_engine('bigquery://bespoke-financial/ProdMetrcData')

logging.basicConfig(format='%(asctime)s [%(levelname)s] - %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

def _reload_libs():
    importlib.reload(util)
    importlib.reload(valuations_util)
    importlib.reload(download_util)

%load_ext autoreload
%autoreload 2

In [ ]:
COMPANY_NAME = 'CG'
COMPANY_IDENTIFIER = 'CG'
LICENSE_NUMBERS = ['C9-0000157-LIC']
TRANSFER_PACKAGES_START_DATE = '2019-01-01'
SALES_TRANSACTIONS_START_DATE = '2019-01-01'
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0,
    'find_parent_child_relationships': False,
    'use_prices_to_fill_missing_incoming': False,
    'external_pricing_data_config': {
        'category_to_fixed_prices': {
            'Buds': {
                'grams': 10.0
            },
            'Infused (edible)': {
                'each': 4.0
            },
            'Infused (non-edible)': {
                'each': 3.0
            },
            'Vape Product': {
                'each': 3.0
            },
            'Concentrate (Bulk)': {
                'grams': 6.0
            },
            'Concentrate': {
                'grams': 7.0
            },
            'Raw Pre-Rolls': {
                'grams': 7.0,
                'pounds': 80.0
            },
            'Shake/Trim (by strain)': {
                'grams': 8.0
            }
        }
    },
    'use_margin_estimate_config': False,
    'margin_estimate_config': {
        'category_to_margin_estimate': {
            'Buds': 0.4267,
            'Infused (edible)': 0.4953,
            'Infused (non-edible)': 0.4183,
            'Vape Product': 0.43,
            'Concentrate (Bulk)': 0.46,
            'Concentrate': 0.46,
            'Raw Pre-Rolls': 0.485,
            'Shake/Trim (by strain)': 0.4267
        }
    },
    'cogs_analysis_params': {
        'readjust_profit_threshold': 0.90, # 0.9
        'readjust_type': 'adjust' # adjust, remove
    }
}
TODAY_DATE = date.today()
print('Today is {}'.format(TODAY_DATE))

In [ ]:
# Download packages, sales transactions, incoming / outgoing tranfers
q = download_util.Query(
    inventory_dates=[],
    transfer_packages_start_date=TRANSFER_PACKAGES_START_DATE,
    sales_transactions_start_date=SALES_TRANSACTIONS_START_DATE,
    company_identifier=COMPANY_IDENTIFIER,
    company_name=COMPANY_NAME,
    license_numbers=LICENSE_NUMBERS,
)

all_dataframes_dict = download_util.get_dataframes_for_analysis(
    q, engine, dry_run=False)

In [ ]:
q.inventory_dates = download_util.get_inventory_dates(
    all_dataframes_dict, TODAY_DATE)
d = util.Download()
sql_helper = util.BigQuerySQLHelper(engine)
d.download_dataframes(
    all_dataframes_dict=all_dataframes_dict,
    sql_helper=sql_helper
)

In [ ]:
id_to_history = util.get_histories(d, ANALYSIS_PARAMS)
util.print_counts(id_to_history)
# counts_analysis_dict = util.create_inventory_xlsx(
#     d, id_to_history, q, params=ANALYSIS_PARAMS)

In [ ]:
computed_resp = util.compute_inventory_across_dates(
    d, q.inventory_dates, ANALYSIS_PARAMS, using_nb=True
)

today_date_str = TODAY_DATE.strftime('%m/%d/%Y')
#INVENTORY_DATES = [today_date_str]
compare_inventory_res = util.compare_computed_vs_actual_inventory(
    computed=computed_resp['date_to_computed_inventory_dataframe'][today_date_str],
    actual=d.inventory_packages_dataframe,
    compare_options={
        'num_errors_to_show': 10,
        'accept_computed_when_sold_out': True
    }
)


In [ ]:
print(f'Plotting sales revenue vs cost-based inventory valuation for dates: {q.inventory_dates}')
valuations_util.plot_inventory_and_revenue(
    q=q,
    sales_receipts_dataframe=d.sales_receipts_dataframe,
    inventory_valuations=computed_resp['inventory_valuations']
)

In [ ]:
# For debugging individual package histories
PACKAGE_IDS = [
  '19300875'
]

util.analyze_specific_package_histories(
    d, PACKAGE_IDS, params=ANALYSIS_PARAMS)